In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters=1000
learning_rate=3e-2
eval_iters = 250

cpu


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
83


In [4]:
str_to_int = {c:i for i,c in enumerate(chars)}
int_to_str = {i:c for i,c in enumerate(chars)}
encode = lambda s: [str_to_int[i] for i in s]
decode = lambda l: ''.join([int_to_str[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([82, 46, 64, 75, 67, 60, 24,  1, 30, 70, 73, 70, 75, 63, 80,  1, 56, 69,
        59,  1, 75, 63, 60,  1, 49, 64, 81, 56, 73, 59,  1, 64, 69,  1, 41, 81,
         0,  0, 27, 76, 75, 63, 70, 73, 24,  1, 38, 12,  1, 32, 73, 56, 69, 66,
         1, 28, 56, 76, 68,  0,  0, 35, 67, 67, 76, 74, 75, 73, 56, 75, 70, 73,
        24,  1, 36, 70, 63, 69,  1, 44, 12,  1, 40, 60, 64, 67, 67,  0,  0, 44,
        60, 67, 60, 56, 74, 60,  1, 59, 56, 75])


In [5]:
n = int(.8 * len(data))
train_data = data[:n]
validate_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else validate_data
    xi = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:block_size+i] for i in xi])
    y = torch.stack([data[i+1:block_size+1+i] for i in xi])
    x, y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('predictive vals:',x)
print('target:', y)


predictive vals: tensor([[64, 60, 59,  1, 70, 76, 75,  1],
        [60, 73, 78, 56, 73, 59,  1, 70],
        [56, 62, 64, 58, 12,  3,  0,  0],
        [80, 10,  1, 56, 74,  1, 80, 70]])
target: tensor([[60, 59,  1, 70, 76, 75,  1, 64],
        [73, 78, 56, 73, 59,  1, 70, 76],
        [62, 64, 58, 12,  3,  0,  0, 53],
        [10,  1, 56, 74,  1, 80, 70, 76]])


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval();
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
        

In [7]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([82]) target is tensor(46)
when input is tensor([82, 46]) target is tensor(64)
when input is tensor([82, 46, 64]) target is tensor(75)
when input is tensor([82, 46, 64, 75]) target is tensor(67)
when input is tensor([82, 46, 64, 75, 67]) target is tensor(60)
when input is tensor([82, 46, 64, 75, 67, 60]) target is tensor(24)
when input is tensor([82, 46, 64, 75, 67, 60, 24]) target is tensor(1)
when input is tensor([82, 46, 64, 75, 67, 60, 24,  1]) target is tensor(30)


In [9]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    def generate(self, index, max_tokens):
        for _ in range(max_tokens):
            logits, loss = self.forward(index)
            print(logits)
            logits = logits[:,-1,:]
            
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=-1)
        return index
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(m.generate(context, max_tokens=500)[0].tolist())
print(generated_chars)

tensor([[[ 0.5580,  2.6284,  1.2398, -0.8629,  1.5941,  0.6498, -1.3693,
          -0.1957, -1.6548, -0.2219,  0.1495, -0.3402,  1.7308, -0.3259,
           0.9047,  2.1052,  0.5992,  0.4183,  0.5133, -0.7826, -0.3476,
           0.6523,  0.9071,  0.1080,  1.8245,  0.4611, -0.1969, -0.4660,
          -0.8643, -0.3104, -0.5975,  0.8079,  1.7924,  0.3035, -0.1527,
           0.4844, -0.2486,  0.1546,  1.0650, -0.3063, -1.4770, -0.9785,
           1.9472, -2.1936, -1.4096, -0.4657, -0.1345,  0.1386, -1.6804,
          -0.4526,  0.7326, -1.5997,  0.7159, -0.4596,  1.1672,  0.5259,
          -0.5523, -0.5105,  1.7180,  0.2380, -1.4946,  0.2642, -0.6251,
           0.2877, -0.3596, -0.8481,  0.9577,  0.1418,  1.0274,  1.4819,
           0.8760, -0.2086, -0.1339, -0.5646,  0.9635, -0.8379, -0.3999,
          -0.9452, -1.0942,  1.0413,  0.6667, -1.3185,  0.0968]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[ 0.5580,  2.6284,  1.2398, -0.8629,  1.5941,  0.6498, -1.3693,
          -0.1957, -

In [10]:
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train_loss: {losses['train']:.3f}, val_loss: {losses['val']:.3f}")
    #get a batch of sample data
    xb,yb = get_batch('train')
    
    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train_loss: 4.936, val_loss: 4.941
step: 250, train_loss: 2.662, val_loss: 2.708
step: 500, train_loss: 2.549, val_loss: 2.607
step: 750, train_loss: 2.520, val_loss: 2.585
2.067059278488159


In [11]:
context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(m.generate(context, max_tokens=500)[0].tolist())
print(generated_chars)


haco fre intoVareshe warcawit y wofthatower gaswarof Mds cony Ze buthanillenatheresererd f o on Vas th bld isuskind th s, irfrn; g gy ro wok d hire so the vend my tos h pputhe tat pein Eved cathoowna, the mawexhir wirderat mo e thive r  d, thougly Witheaur abowhon trglas,Q,m tth hir herand hetod aghof bon b."


cacodat an Witheth's I't Ma nd

Norot apt, the ca cgetwairer aton.
"

"Bugosh.
"uthi; bund un I
br b-be t?" sh
" waizasthind, HYowind " wede e aine wem gricuas Bur ces es winokernt THorot
